<h1 align="center" style="color:green;font-size: 3em;">
Implementing Fine-tuning Techniques</h1>


Implementing various fine-tuning methods as described in different papers, specifically LoRA and IA3.

We will do this in 3 parts:

Pt1:

In this notebook, we will:

- Evaluate the perplexity of a causal language model.
- Fine-tune a sequence classification model using different learning rates and analyze its performance.